In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import os
import scipy
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from IPython.display import clear_output

# fonctions utiles

In [10]:
def Linearize_audio(aud_data):
    """
    Cette fonction permet de fusionner deux entrées (enregistrement stéréo) en une seule si ce n'est pzs déhà un enregistrement mono
    """
    if len(aud_data.shape)>1:
        linear_audio = (aud_data[:,0]+aud_data[:,1])/2
    else:
        linear_audio = aud_data
    
    linear_audio = np.array(linear_audio)

    msk = (abs(linear_audio[:])>max(linear_audio)*0.1)

    return linear_audio[msk]

def Spectrum_Treatment(data,cut=8):
    """
    Transformée de Fourier, la variable cut permet d'isoler la partie intéressante du spectre
    """

    N = len(data)
    Y = np.fft.fft(data)
    Y = abs(Y[:N//cut])
    return Y

def Accuracy(Nw,X,Y):
    predictions = Nw.forward(X)
    return round(np.mean((Y -predictions)**2 <= (0.05)**2),2) + 0.01*round(np.mean((Y -predictions)**2 <= (0.02)**2),2) #, np.mean([round(Y[i],1)==round(predictions[i],1) for i in range(len(Y))])

def minuscules(String):
    m1,m2 =ord('A'),ord('a')
    dm = m2-m1
    FS = ''
    for s in String:
        if ord(s)<m2:
            FS = FS+chr(ord(s)+dm)
        else:
            FS = FS +s
    return FS

def Reverse(label):
    for i in range(len(classes)):
        if classes[i]==label:
            return i

# Le gros du traitement

In [11]:
#Dimension parameters
NbSegments = 50
FreqHeight = 40
#la partie qui se superpose
Super = 0.5

In [ ]:
DATA = []
LABELS = []

i = 0
cut = 8

IN = "Audio_Trainer" # dossier contenant les données

for file in os.listdir(IN):
    
    rate,aud_data = scipy.io.wavfile.read(IN+"/"+file)
    aud_data = Linearize_audio(aud_data)

    N = len(aud_data)

    Seglen =N//NbSegments
    Super_Seglen = round(Super*N)//NbSegments

    Segments = [aud_data[i*Seglen:(i+1)*(Seglen)+Super_Seglen]  for i in range(NbSegments-1)] #if i<NbSegments-1 else aud_data[i*Seglen:(i+1)*(Seglen)] 
    Segments = np.array(Segments)

    Ys = np.array([Spectrum_Treatment(data,cut) for data in Segments])

    Max = max([max(y) for y in Ys])
    Ys/=Max

    if Ys.shape[1]<FreqHeight :
        i+=1
        print('passed one more : ',i)
        continue

    Seglen = len(Ys[0])//FreqHeight
    Lines = np.array([[np.max(Y[i*Seglen:(i+1)*Seglen]) for i in range(FreqHeight)] for Y in Ys]).T
    """
    plt.imshow(Lines[::-1,:],cmap="gray")
    plt.title(minuscules(file.split()[0]))
    plt.savefig("Spectrum_Trainer/"+file[:-3] +"png",bbox_inches='tight')
    plt.close()
    """
    DATA.append(Lines.flatten())
    LABELS.append(minuscules(file.split()[0]))


DATA = np.array(DATA)

C:\Users\arthu\AppData\Local\Temp\ipykernel_19644\4161505000.py:11: WavFileWarning: Chunk (non-data) not understood, skipping it.
  rate,aud_data = scipy.io.wavfile.read(IN+"/"+file)


passed one more :  1
passed one more :  2


In [5]:
#un bout de code bien compliqué pour obtenir les bons labels

classes = []
for l in LABELS:
    if l not in classes:
        classes.append(l)

Numeric_classes = [i for i,j in enumerate(classes)]

Labels = np.array([Reverse(l) for l in LABELS])

In [6]:
X_train,X_test,Y_train, Y_test = train_test_split(DATA, Labels, test_size=0.2)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

# Utilisation ici de tensorflow

La documentation est très complète, pour faire qimple on utilise ici le modèle séquentiel pour faire l'ANN puis on l'entraine

In [ ]:
import tensorflow as tf

In [ ]:
# utilisation du réseau neuronal,  une configuration peut être {'epochs': 1000, 'First Node': ['relu', 512], 'Second Node': ['relu', 64], 'Third Node': ['tanh', 32.0]}

inputs = tf.keras.Input(shape=(DATA.shape[1],))
x = tf.keras.layers.Dense(256,activation='tanh')(inputs)
x = tf.keras.layers.Dense(128,activation='tanh')(x)
x = tf.keras.layers.Dense(64,activation='tanh')(x)
outputs=tf.keras.layers.Dense(len(classes),activation=tf.nn.softmax)(x)

model = tf.keras.Model(inputs=inputs,outputs=outputs)

# on trouve aussi souvent l'optimizer Adam

model.compile(loss = "sparse_categorical_crossentropy",optimizer="SGD")

NameError: name 'tf' is not defined

In [8]:
history = model.fit(X_train,Y_train,epochs=1000,verbose = 0)

NameError: name 'model' is not defined

In [9]:
predictions = model.predict(X_test)

for i in range(len(X_test)):
    print(classes[np.argmax(predictions[i])],classes[Y_test[i]])

NameError: name 'model' is not defined

In [ ]:
# La "précision du réseau"

print(np.mean([classes[np.argmax(predictions[i])]==classes[Y_test[i]] for i in range(len(X_test))]))